In [1]:
from mbdvv import app, get_solids
from pymbd import MBDCalc, from_volumes, ang, get_kgrid, vdw_params

import numpy as np
import pandas as pd
import os
from itertools import product
from functools import partial
from pkg_resources import resource_stream

In [2]:
solid_data = get_solids(app.ctx)
solid_params = pd.read_csv(resource_stream('mbdvv', 'data/solids.csv')).set_index('label')

In [3]:
def get_last(obj):
    if not isinstance(obj, list):
        return obj
    assert len(obj) == 2
    return obj[-1]

def listify(obj):
    if isinstance(obj, list):
        return obj
    return [obj]

In [4]:
class NegativeEigs(Exception):
    pass


def mbd_rsscs(mbd_calc, coords, alpha_0, C6, R_vdw, beta, lattice=None, k_grid=None):
    def _array(obj, *args, **kwargs):
        if obj is not None:
            return np.array(obj, *args, **kwargs)

    coords = _array(coords, dtype=float, order='F')
    alpha_0 = _array(alpha_0, dtype=float)
    C6 = _array(C6, dtype=float)
    R_vdw = _array(R_vdw, dtype=float)
    freq, freq_w = mbd_calc.omega_grid
    omega = 4./3*C6/alpha_0**2
    alpha_dyn = alpha_0/(1+(freq[:, None]/omega)**2)
    alpha_dyn_rsscs = np.empty_like(alpha_dyn)
    for a, a_scr in zip(alpha_dyn, alpha_dyn_rsscs):
        sigma = (np.sqrt(2./np.pi)*a/3)**(1./3)
        a_nlc = np.linalg.inv(
            np.diag(np.repeat(1./a, 3)) + mbd_calc.dipole_matrix(
                coords, 'fermi,dip,gg', sigma=sigma, R_vdw=R_vdw, beta=beta, lattice=lattice,
            )
        )
        a_scr[:] = sum(a_nlc[i::3, i::3].sum(1) for i in range(3))/3
    C6_rsscs = 3./np.pi*np.sum(freq_w[:, None]*alpha_dyn_rsscs**2, 0)
    R_vdw_rsscs = R_vdw*(alpha_dyn_rsscs[0, :]/alpha_0)**(1./3)
    omega_rsscs = 4./3*C6_rsscs/alpha_dyn_rsscs[0, :]**2
    pre = np.repeat(omega_rsscs*np.sqrt(alpha_dyn_rsscs[0, :]), 3)
    if lattice is None:
        k_grid = [None]
    else:
        assert k_grid is not None
        k_grid = get_kgrid(lattice, k_grid)
    ene = 0
    for k_point in k_grid:
        eigs = np.linalg.eigvalsh(
            np.diag(np.repeat(omega_rsscs**2, 3)) +
            np.outer(pre, pre)*mbd_calc.dipole_matrix(
                coords, 'fermi,dip', R_vdw=R_vdw_rsscs, beta=beta, lattice=lattice, k_point=k_point
            )
        )
        if np.any(eigs < 0):
            raise NegativeEigs(eigs, k_point)
        ene += np.sum(np.sqrt(eigs))/2-3*np.sum(omega_rsscs)/2
    ene /= len(k_grid)
    return ene

In [5]:
%%time
with MBDCalc() as mbd_calc:
    for (label, scale), data in solid_data['solids'].loc[(slice(None), 1.),].iloc[:10].itertuples():
        coords = data['coords']['value'].T
        species = listify(data['elems']['atom'])
        lattice = data['lattice_vector']['value']
        pbe_ene = data['energy'][0]['value'][0]
        volumes = get_last(data['volumes'])
        alpha_vv = get_last(data['vv_pols'])
        free_atoms = get_last(data['free_atoms'])
        species_idx = free_atoms['species']-1
        volumes_free = free_atoms['volumes'][species_idx]
        alpha_vv_free = free_atoms['vv_pols'][:, species_idx]
        freq_w = get_last(data['omega_grid_w'])
        
        alpha_0, C6, R_vdw = from_volumes(species, volumes/volumes_free)
        alpha_0_free = np.array([vdw_params.get(sp)['alpha_0'] for sp in species])
        C6_vv = 3/np.pi*np.sum(freq_w[:, None]*alpha_vv**2, 0)
        C6_vv_free = 3/np.pi*np.sum(freq_w[:, None]*alpha_vv_free**2, 0)
        R_vdw_1 = 2.5*alpha_0**(1/7)
        R_vdw_vv = 2.5*alpha_vv[0]**(1/7)
        k_grid = np.repeat(6, 3)
        try:
            ene = mbd_rsscs(
                mbd_calc,
                coords,
                alpha_0, C6, R_vdw,
                0.83,
                lattice=lattice,
                k_grid=k_grid
            )
        except NegativeEigs as e:
            ene = e.args[0]
        except np.linalg.LinAlgError as e:
            ene = 'did not converge'
        print(label, scale, ene)
        # print(label, scale, alpha_0, alpha_vv[0])
        # print(label, scale, vdw_params.get(label)['alpha_0'], alpha_vv_free[0])
        # print(label, scale, alpha_0, alpha_vv[0]*alpha_0_free/alpha_vv_free[0])
        # print(label, scale, R_vdw, R_vdw_1)

Ag 1.0 [-0.0075608  -0.0075608   0.23444993]
Al 1.0 [-0.00833541 -0.00833541  0.38088662]
Au 1.0 -0.017786694157280677
Ca 1.0 [-0.00967489 -0.00967489  0.1101537 ]
Cu 1.0 [-0.00971144 -0.00971144  0.36980835]
Ir 1.0 [-0.00580345 -0.00580345  0.53573045]
Ni 1.0 [-0.01810629 -0.01810629  0.55214333]
Pb 1.0 -0.0172857723441398
Pd 1.0 -0.012230739064077248
Pt 1.0 [-0.00339218 -0.00339218  0.57293716]
CPU times: user 1.12 s, sys: 77.3 ms, total: 1.2 s
Wall time: 2.08 s
